# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import  create_engine, inspect

import re
import nltk
nltk.download(['punkt','wordnet','stopwords','averaged_perceptron_tagger'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Y_Han\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Y_Han\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Y_Han\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Y_Han\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql(
    "SELECT * FROM DisasterResponse",
    con=engine
)

X = df["message"]
Y = df.drop(columns=["id", "message", "original", "genre"]).astype('bool')

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002159BB15820>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
print(classification_report(y_test, y_pred, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4991
               request       0.82      0.48      0.60      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.69      0.72      2670
          medical_help       0.72      0.09      0.16       535
      medical_products       0.78      0.08      0.15       344
     search_and_rescue       0.56      0.03      0.06       159
              security       0.20      0.01      0.02       116
              military       0.78      0.09      0.16       200
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.39      0.54       418
                  food       0.85      0.61      0.71       745
               shelter       0.80      0.39      0.53       581
              clothing       0.71      0.10      0.18        98
                 money       0.80      

C:\Users\Y_Han\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Y_Han\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Y_Han\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Y_Han\anaconda3\lib\site-packages\sklearn\metrics\_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
    'clf__estimator__n_estimators': [50, 100, 150],
    'clf__estimator__criterion': ["gini", "entropy"]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002809D5F6C10>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__criterion': ['gini', 'entropy'],
                         'clf__estimator__n_estimators': [50, 100, 150]})

In [11]:
y_pred2 = pipeline.predict(X_test)

In [12]:
print(classification_report(y_test, y_pred2, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      4991
               request       0.84      0.50      0.62      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.70      0.72      2670
          medical_help       0.64      0.08      0.14       535
      medical_products       0.79      0.08      0.14       344
     search_and_rescue       0.62      0.06      0.11       159
              security       0.00      0.00      0.00       116
              military       0.57      0.06      0.11       200
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.34      0.49       418
                  food       0.86      0.58      0.70       745
               shelter       0.81      0.41      0.55       581
              clothing       0.88      0.07      0.13        98
                 money       0.80      

C:\Users\Y_Han\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Y_Han\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Y_Han\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Y_Han\anaconda3\lib\site-packages\sklearn\metrics\_cla

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [9]:
modelname = 'disaster_response_model'
# pickle.dump(cv.best_estimator_, open(modelname, 'wb'))
pickle.dump(pipeline, open(modelname, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.